In [1]:
import gym
import numpy as np
import random

In [2]:
env = gym.make('SpaceInvaders-ram-v0')

In [3]:
actions = env.action_space.n

In [4]:
env.observation_space.shape

(128,)

In [5]:
# episodes = 5
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0
    
#     while not done:
#         env.render()
#         action = random.choice([0,1,2,3,4,5])
#         state, reward, done, info = env.step(action)
#         score += reward
#     print(f'Episode: {episode} Score: {score}')
# env.close()

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [11]:
del model

In [12]:
model = Sequential()
model.add(Flatten(input_shape=(5,) + env.observation_space.shape))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(256, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(actions, activation='linear'))

In [8]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory, Memory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy, BoltzmannQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy

In [128]:
1/(np.sqrt(10000/6))

0.02449489742783178

In [9]:
def build_agent(model, actions, c, n):
    policy = BoltzmannGumbelQPolicy(c)
    memory = SequentialMemory(limit=n, window_length=5)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   test_policy=BoltzmannQPolicy(1/(np.sqrt(n/actions))),
                   enable_dueling_network=True, dueling_type='avg', nb_actions=actions, 
                   nb_steps_warmup=1000)
    return dqn

In [13]:
dqn = build_agent(model, 6, 2, 100000)

In [14]:
dqn.compile(optimizer='adam')

In [24]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)
scores = dqn.test(env, nb_episodes=5, visualize=True)
print('Mean reward: ',np.mean(scores.history['episode_reward']))
env.close()
iterations += 1e5
print(iterations)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 152s 15ms/step - reward: 0.2420
12 episodes - episode_reward: 186.250 [35.000, 600.000] - loss: 6.039 - mean_q: 52.008 - ale.lives: 2.106

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 164s 16ms/step - reward: 0.2870
10 episodes - episode_reward: 299.500 [185.000, 600.000] - loss: 6.197 - mean_q: 52.132 - ale.lives: 2.076

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 166s 17ms/step - reward: 0.2155
13 episodes - episode_reward: 158.462 [60.000, 345.000] - loss: 7.137 - mean_q: 52.335 - ale.lives: 1.861

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 166s 17ms/step - reward: 0.2960
12 episodes - episode_reward: 259.583 [75.000, 470.000] - loss: 7.512 - mean_q: 52.309 - ale.lives: 2.032

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 166s 17

In [ ]:
dqn.save_weights('./saved_weights/dqn-bge.h5f', overwrite=True)

In [62]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 5 episodes ...
Episode 1: reward: 180.000, steps: 876
Episode 2: reward: 210.000, steps: 846
Episode 3: reward: 80.000, steps: 529
Episode 4: reward: 105.000, steps: 655
Episode 5: reward: 260.000, steps: 977
167.0


In [23]:
iterations = 300000